In [39]:
import multiprocessing as mp
import pyupbit
import datetime    
import sqlite3

# 데이터 베이스에 저장
def save_to_database(con, data): 

    #cursor = con.cursor()는 SQLite 데이터베이스에서 
    cursor = con.cursor() 

    code = data['code']
    open_price = data['opening_price']
    high_price = data['high_price']
    low_price = data['low_price']
    now_price = data['trade_price']
    prev_close = data['prev_closing_price']
    change = data['change']
    change_price = data['signed_change_price']
    ts = data['trade_timestamp']

    dt = datetime.datetime.fromtimestamp(ts/1000)
    
    # SQLite에 데이터 삽입하기 위한 쿼리. 'ticker_data'란 테이블에 데이터 삽입
    query = """
    INSERT OR IGNORE INTO ticker_data (datetime, code, open, high, low, now, prev_close, change, change_pr)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    # 커서를 사용하여 SQL 쿼리를 실행. 첫 번째 매개변수로는 실행할 쿼리, 두 번째 매개변수로는 실제 데이터를 전달
    cursor.execute(query, (dt, code, open_price, high_price, low_price, now_price, prev_close, change, change_price))

    # 데이터베이스에 대한 변경 사항을 저장. 데이터 추가,수정,삭제할 때 커밋해야됨.

    con.commit()


if __name__ == "__main__":
    krw_tickers = pyupbit.get_tickers(fiat="KRW-BTC") # Upbit에서 거래되는 비트코인(KRW-BTC)과 관련된 티커(마켓 코드)
    queue = mp.Queue() # 멀티프로세싱을 위한 큐를 생성. 웹소켓 프로세스에서 메인 프로세스로 데이터를 전달하기 위해.
    proc = mp.Process( # 멀티프로세스로 실행할 웹소켓 클라이언트 프로세스를 생성. 이 프로세스는 Upbit의 WebSocket을 사용하여 실시간 거래 정보를 가져오고, 큐에 해당 정보를 넣는다.
        target=pyupbit.WebSocketClient,
        args=('ticker', krw_tickers, queue),
        daemon=True
    )
    proc.start()

    # SQLite 데이터베이스 연결
    con = sqlite3.connect("fin.db") #SQLite 데이터베이스(fin.db)에 연결

    while True: # 무한 루프를 통해 계속해서 실시간 데이터를 가져오고 저장
        data = queue.get() #: 큐에서 데이터를 가져온다. 이는 웹소켓 프로세스에서 메인 프로세스로 데이터를 전달하기 위한 메커니즘

        code = data['code'] # 마켓 코드 (비트코인만)
        open_price = data['opening_price'] # 시가
        high_price = data['high_price']    # 고가
        low_price = data['low_price']      # 저가   
        now_price = data['trade_price']    # 현재가
        prev_close = data['prev_closing_price'] # 전일 종가
        change = data['change']            # 전일대비 상승,보합,하락
        change_price = data['signed_change_price']  # 전일 대비 값
        ts = data['trade_timestamp'] # 체결 타임스탬프 (milliseconds)

        dt = datetime.datetime.fromtimestamp(ts/1000) # 00:00:00 (UTC)를 기준으로 경과한 시간을 초 단위로 나타낸 것
        
        # data 화면에 출력
        print(dt, code, open_price, high_price, low_price, now_price, prev_close, change, change_price)

        # SQLite 데이터베이스에 저장하는 함수 호출
        save_to_database(con, data)
    con.close() # 반복문 끝나면 데이터베이스 닫아야 함 -> 안 닫으면 다시 실행했을 때 database가 locked 될 수도

## 웹소켓을 이용해 업비트에서 티커, 시가, 고가, 저가, 현재가 등을 출력하고
## 해당 값들을 DB browser SQLite에 저장하는 프로그램

2023-12-17 02:20:24.914000 KRW-BTC 58020000.0 58562000.0 57574000.0 58326000.0 58028000.0 RISE 298000.0
2023-12-17 02:20:24.914000 KRW-BTC 58020000.0 58562000.0 57574000.0 58326000.0 58028000.0 RISE 298000.0
2023-12-17 02:20:26.549000 KRW-BTC 58020000.0 58562000.0 57574000.0 58326000.0 58028000.0 RISE 298000.0
2023-12-17 02:20:28.564000 KRW-BTC 58020000.0 58562000.0 57574000.0 58326000.0 58028000.0 RISE 298000.0
2023-12-17 02:20:29.228000 KRW-BTC 58020000.0 58562000.0 57574000.0 58326000.0 58028000.0 RISE 298000.0
2023-12-17 02:20:29.395000 KRW-BTC 58020000.0 58562000.0 57574000.0 58326000.0 58028000.0 RISE 298000.0
2023-12-17 02:20:29.621000 KRW-BTC 58020000.0 58562000.0 57574000.0 58326000.0 58028000.0 RISE 298000.0
2023-12-17 02:20:29.621000 KRW-BTC 58020000.0 58562000.0 57574000.0 58326000.0 58028000.0 RISE 298000.0
2023-12-17 02:20:30.342000 KRW-BTC 58020000.0 58562000.0 57574000.0 58326000.0 58028000.0 RISE 298000.0
2023-12-17 02:20:31.414000 KRW-BTC 58020000.0 58562000.0 5757400

KeyboardInterrupt: 